### Task 3
Calculate the average cases per day from the beginning to the end of the dataset. Use .sum to obtain the total number of cases and then use .nunique to determine the number of days in the dataset. The average cases per day can be calculated by dividing the total number of cases by the number of days.

In [117]:
import pandas as pd

excel_file_path = "Labb-data/Folkhalsomyndigheten_Covid19.xlsx"


df = pd.read_excel(excel_file_path, sheet_name=0)
total_cases = df["Totalt_antal_fall"].sum()
print(f"Total cases: {total_cases}")

total_days = df["Statistikdatum"].nunique()
print(f"Total days in dataset: {total_days}")

average_cases_per_day = int(total_cases / total_days)
print(f"Average number of cases per day: {average_cases_per_day}")

Total cases: 2611580
Total days in dataset: 996
Average number of cases per day: 2622


### Intensive Care Unit (ICU) or Intensivvård (IVA) in swedish
The line chart depicts periods when individuals were admitted to the ICU/IVA. The purpose of the data is to gain an overview of mortality rates.

In [154]:
import plotly.express as px

excel_file_path = "Labb-data/Folkhalsomyndigheten_Covid19.xlsx"

df = pd.read_excel(excel_file_path, sheet_name="Antal avlidna per dag")
df_IVA = pd.read_excel(excel_file_path, sheet_name="Antal intensivvårdade per dag")


df.rename(columns={"Datum_avliden": "Datum"}, inplace=True)
df_IVA.rename(columns={"Datum_vårdstart": "Datum"}, inplace=True)

combined_df = pd.concat([df.set_index("Datum"), df_IVA.set_index("Datum")], axis=1)
combined_df = combined_df.reset_index()

fig = px.area(combined_df, x="Datum", 
              y=["Antal_avlidna", "Antal_intensivvårdade"], 
              labels={"value": "Total", "variable": "Deaths and IVA Admissions", "Datum": "Date"},
              title="Total deaths and patients in IVA")

fig.show()
fig.write_html("Assigment3_Mortality.html")

### Filter vaccine data week by week

In [110]:
vaccine_data = pd.read_excel("Labb-data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx", sheet_name="Vaccinationer tidsserie")

vaccine_data["År"] = vaccine_data["År"].astype(str).str.replace(".", "")

vaccine_data = (
    vaccine_data[vaccine_data["Region"] == "| Sverige |"]
    .groupby(["Vecka", "År"])["Antal vaccinationer"]
    .sum()
    .reset_index()
)

print(vaccine_data)


    Vecka    År  Antal vaccinationer
0       1  2021                53258
1       1  2022             18002625
2       2  2021               132915
3       2  2022             18475540
4       3  2021               249712
..    ...   ...                  ...
91     50  2021             17386427
92     51  2021             17615962
93     52  2020                 2478
94     52  2021             17767162
95     53  2020                16581

[96 rows x 3 columns]


### Covid-19 cases and vaccination per week

The goal of comparing Covid-19 cases with vaccination is to understand if the vaccine is effective. By examining the data, we can see a trend, as more people get vaccinated, there appears to be a decrease in weekly Covid-19 cases

In [119]:
vaccine_data = pd.read_excel("Labb-data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx", sheet_name="Vaccinationer tidsserie")
covid_data = pd.read_excel("Labb-data/Folkhalsomyndigheten_Covid19.xlsx", sheet_name="Veckodata Riket")

vaccine_data["År"] = vaccine_data["År"].astype(str).str.replace(".", "")
covid_data["år"] = covid_data["år"].astype(str)

vaccine_data = (
    vaccine_data[(vaccine_data["Region"] == "| Sverige |") & (vaccine_data["År"] == "2021")]
    .groupby(["Vecka", "År"])["Antal vaccinationer"]
    .sum()
    .reset_index()
)

# Rename columns in covid_data to match vaccine_data_sweden
covid_data.rename(columns={"veckonummer": "Vecka"}, inplace=True)

# Merge the DataFrames so both have same names for vecka
merged_data = pd.merge(vaccine_data, covid_data, left_on=["Vecka", "År"], right_on=["Vecka", "år"], how="outer")
merged_data.fillna(0, inplace=True)

fig = px.bar(merged_data.head(5), x="Vecka", y=["Antal vaccinationer", "Antal_fall_vecka"],
             title="Total Covid-19 and total vaccinations for the first five weeks of 2021 (Sweden)",
             labels={"value": "Total", "variable": "Category", "Vecka": "Week"},
             barmode="group",
             template = "plotly_dark")

fig.show()
# fig.write_html("Assigment3_effectiveness")

### Analyzing Patients in ICU (Sweden)
The goal with these three plots is to discern whether there is a pattern among individuals admitted to the Intensive Care Unit (ICU). Upon examining the charts, a clear trend emerges, indicating that the majority of individuals admitted to the ICU are older, with a higher representation of men. Additionally, the charts illustrate that women are slightly more inclined to receive the vaccine, although the difference is not substantial.

In [155]:
vaccine_data = pd.read_excel("Labb-data/Folkhalsomyndigheten_Covid19_vaccine.xlsx", sheet_name="Vaccinerade kön")

fig = px.bar(vaccine_data, x="Vaccinationsstatus", y="Antal vaccinerade", color="Kön",
             title="Vaccination by Gender",
             labels={"Antal vaccinerade": "Number of Vaccinated"},
             barmode="group",
             template="plotly")

fig.show()
# fig.write_html("Task3_Gender.html")

In [156]:
vaccine_data = pd.read_excel("Labb-data/Folkhalsomyndigheten_Covid19.xlsx", sheet_name="Totalt antal per kön")

fig = px.pie(vaccine_data, names="Kön", values="Totalt_antal_intensivvårdade",
             title="Total Intensive Care Admissions by Gender")

fig.show()
# fig.write_html("Task3_Gender_pie.html")

In [157]:
vaccine_data = pd.read_excel("Labb-data/Folkhalsomyndigheten_Covid19.xlsx", sheet_name="Totalt antal per åldersgrupp")

fig = px.bar(vaccine_data, x="Åldersgrupp", y="Totalt_antal_intensivvårdade",
             title="Intensive Care Admissions by Age Group",
             labels={"Totalt_antal_intensivvårdade": "Number of IVA Admissions", "Åldersgrupp": "Age group"},
             template="plotly_dark")

fig.show()
# fig.write_html("Task3_Age.html")